In [3]:
import requests

url = "https://audio.dubverse.ai/api/tts"
# मैं तुम्हारे बिना अधूरी हूँ। मैं तुम्हारे साथ सोना चाहती हूँ—not सिर्फ जिस्म से, बल्कि रूह से जुड़ने के लिए। तुम्हारे बिना... मैं जैसे धीरे-धीरे मर रही हूँ
payload = {
    "text": """
   एक छोटे से गाँव में, रात के अंधेरे में एक अजीब सी चीख सुनाई दी। गाँव वाले डर गए, क्योंकि ये चीखें अक्सर गाँव के बाहर स्थित पुराने, परित्यक्त किले से आती थीं।

एक युवा लड़की, राधि। एक रात, वह किले की ओर चल पड़ी, उसके दिल में डर और जिज्ञासा दोनों थे।
    """,
    "speaker_no": 1252,
    "config": {
        "use_streaming_response": False,
        # "sample_rate": 22050,
        # "instructions": "<string>"
    },
    # "callback_url": "<string>"
}
headers = {
    "X-API-KEY": "cnfCWZhUoraJuWucivbaoX8N01MawsIM",
    "Content-Type": "application/json"
}

response = requests.request("POST", url, json=payload, headers=headers)
response

<Response [200]>

In [4]:
import time
with open(f"{time.strftime('%Y-%m-%d_%H-%M-%S')}.mp3", 'wb') as f:
    f.write(response.content)


In [6]:
# !pip install elevenlabs

In [9]:

import os
import uuid
from elevenlabs import VoiceSettings
from elevenlabs.client import ElevenLabs

ELEVENLABS_API_KEY = os.getenv("ELEVENLABS_API_KEY")
client = ElevenLabs(
    api_key=ELEVENLABS_API_KEY,
)


def text_to_speech_file(text: str) -> str:
    # Calling the text_to_speech conversion API with detailed parameters
    response = client.text_to_speech.convert(
        voice_id="MF4J4IDTRo0AxOO4dpFR", # Adam pre-made voice
        output_format="mp3_22050_32",
        text=text,
        model_id="eleven_turbo_v2_5", # use the turbo model for low latency
        # Optional voice settings that allow you to customize the output
        voice_settings=VoiceSettings(
            stability=0.0,
            similarity_boost=1.0,
            style=0.0,
            use_speaker_boost=True,
            speed=1.0,
        ),
    )

    # uncomment the line below to play the audio back
    # play(response)

    # Generating a unique file name for the output MP3 file
    save_file_path = f"{uuid.uuid4()}.mp3"

    # Writing the audio to a file
    with open(save_file_path, "wb") as f:
        for chunk in response:
            if chunk:
                f.write(chunk)

    print(f"{save_file_path}: A new audio file was saved successfully!")

    # Return the path of the saved audio file
    return save_file_path



In [10]:
text_to_speech_file("एक छोटे गाँव में, राजू नाम का एक लड़का रहता था। राजू को पेड़-पौधों से बहुत प्यार था। एक दिन, उसने एक छोटा सा पौधा लगाया। वह हर रोज उसे पानी देता और उसकी देखभाल करता। धीरे-धीरे, पौधा बड़ा हो गया और एक सुंदर पेड़ बन गया। राजू बहुत खुश था।")

10f5e838-0610-4287-8969-392fca1f6ea8.mp3: A new audio file was saved successfully!


'10f5e838-0610-4287-8969-392fca1f6ea8.mp3'

In [8]:
import boto3

import os
from dotenv import load_dotenv
load_dotenv()
aws_key_id = os.getenv("AWS_ACCESS_KEY")
aws_secret_id = os.getenv("AWS_SECRET_KEY")

s3 = boto3.client(
    's3',
    aws_access_key_id=aws_key_id,
    aws_secret_access_key= aws_secret_id
)


In [12]:
import boto3
# Retrieve the list of existing buckets
s3 = boto3.client('s3')
response = s3.list_buckets()

# Output the bucket names
print('Existing buckets:')
for bucket in response['Buckets']:
    print(f'  {bucket["Name"]}')

Existing buckets:
  story-agent


In [15]:
response = s3.list_objects_v2(Bucket="story-agent")

if 'Contents' in response:
    print("Files in bucket:")
    for obj in response['Contents']:
        print(obj['Key'])
else:
    print("Bucket is empty or does not exist.")

Files in bucket:
ElevenLabs/2025-04-25_20-10-27.mp3
ElevenLabs/2025-04-26_00-19-29.mp3
ElevenLabs/2025-04-26_00-31-12.mp3
ElevenLabs/2025-04-26_00-39-36.mp3
ElevenLabs/2025-04-26_00-42-03.mp3
